In [1]:
import torch

/home/arthur/anaconda3/envs/peract/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [31]:

a = torch.randn(1, 1, 10, 10, 10)

In [19]:
import torch

# Example tensor A and indices
b, c, h, w, d = 1, 2, 4, 4, 4  # Adjust dimensions as needed
A = torch.rand(b, c, h, w, d)  # Example tensor

# Example indices with shape (b, c, 100, 3)
# Assuming `indices` is the tensor you have, e.g.:
indices = torch.randint(0, 4, (b, c, 5, 3))  # Example indices, adjust as needed

print(indices)
# Value x to set
x = 999999

# Generate indices for the b and c dimensions
b_indices = torch.arange(b).view(b, 1, 1, 1).expand(-1, c, indices.shape[2], 1)
c_indices = torch.arange(c).view(1, c, 1, 1).expand(b, -1, indices.shape[2], 1)

# Use advanced indexing to set values
# Flatten the last three dimensions in A to use with flat indices
A_flat = A.view(b, c, -1)

# Calculate flat indices for the last three dimensions
flat_indices = indices[..., 0]*w*d + indices[..., 1]*d + indices[..., 2]

# Set the values using the generated indices
A_flat[b_indices, c_indices, flat_indices.unsqueeze(-1)] = x

# Reshape A_flat back if needed (it's not actually necessary here, 
# since we're modifying A in place through its view)
A = A_flat.view(b, c, h, w, d)
A


tensor([[[[3, 3, 3],
          [0, 2, 1],
          [2, 2, 2],
          [2, 3, 3],
          [3, 3, 3]],

         [[1, 1, 2],
          [3, 0, 0],
          [0, 2, 2],
          [2, 0, 1],
          [1, 1, 0]]]])


tensor([[[[[6.5909e-01, 9.7190e-01, 1.9391e-01, 1.3816e-01],
           [4.2687e-01, 5.4640e-01, 6.5528e-01, 8.1389e-01],
           [1.7854e-01, 1.0000e+06, 9.0916e-01, 4.1194e-01],
           [4.3824e-01, 5.0802e-01, 1.2147e-02, 3.3010e-01]],

          [[6.5977e-01, 6.0427e-01, 7.7243e-01, 4.0838e-01],
           [9.3267e-01, 6.2912e-01, 6.7900e-01, 3.6344e-01],
           [1.2844e-01, 9.9407e-01, 3.3844e-02, 7.8095e-02],
           [3.0516e-01, 8.9618e-01, 6.6241e-01, 9.4504e-01]],

          [[1.0489e-01, 7.3050e-01, 7.6884e-01, 9.1267e-01],
           [8.4482e-01, 3.6059e-01, 5.7230e-01, 4.4435e-01],
           [9.5619e-01, 6.3366e-01, 1.0000e+06, 2.1029e-02],
           [6.6485e-01, 3.7745e-02, 7.7088e-01, 1.0000e+06]],

          [[3.1593e-02, 4.6744e-01, 1.6832e-01, 5.1431e-01],
           [6.4559e-01, 6.9997e-01, 1.7685e-01, 6.7740e-01],
           [4.6544e-01, 3.6428e-01, 7.5908e-01, 6.0302e-01],
           [9.7668e-01, 8.5747e-01, 2.8905e-01, 1.0000e+06]]],


         [[[4.

In [52]:
def _topk_3d(tensor_orig, k=100):
    b, c, d, h, w = tensor_orig.shape  # c will be one
    idxs = torch.topk(tensor_orig.view(b, c, -1), k, dim=-1).indices
    indices = torch.cat([((idxs // h) // d), (idxs // h) % w, idxs % w], 1).transpose(1, 2).view(b, c, k, 3)
    return indices


top4 = _topk_3d(a, 4)
print(top4.shape,"\n",top4,"\n")


b = torch.arange(a.size(0)).unsqueeze(1).unsqueeze(2)
c = torch.arange(a.size(1)).unsqueeze(0).unsqueeze(2)

print(top4[b,c,:,0],top4[b,c,:,1],top4[b,c,:,2])

# retreive the top 4 indices
print(a[b,c,top4[b,c,:,0],top4[b,c,:,1],top4[b,c,:,2]])


print(a[b,c,top4[b,c,:,2],top4[b,c,:,1],top4[b,c,:,0]])

print([a[0,0,top4[0,0,i,0],top4[0,0,i,1],top4[0,0,i,2]] for i in range(4)])

b = a[:]
for i in range(4):
    print(b.max())
    b =b[b!=b.max()]


torch.Size([1, 1, 4, 3]) 
 tensor([[[[5, 1, 7],
          [9, 5, 5],
          [5, 0, 2],
          [6, 6, 0]]]]) 

tensor([[[[5, 9, 5, 6]]]]) tensor([[[[1, 5, 0, 6]]]]) tensor([[[[7, 5, 2, 0]]]])
tensor([[[[3.2964, 3.1179, 2.7908, 2.7488]]]])
tensor([[[[ 0.2395, -0.0970,  0.8501,  0.6692]]]])
[tensor(3.2964), tensor(3.1179), tensor(2.7908), tensor(2.7488)]
tensor(3.2964)
tensor(3.1179)
tensor(2.7908)
tensor(2.7488)


In [149]:

a = torch.randn(1, 3, 6, 15, 3)
b, c, d, h, w = a.shape 
n = b*c*d*h*w
a = torch.arange(n).reshape(a.shape)
((n-1)%(h*w))%w
((n-1)%(d*h*w))%h
((n-1)%(c*d*h*w))%d

((n-1)%(b*c*d*h*w))%c

(n-1)%w

((n-1)//c)%h




# ((n-1)//(h*w))%h



14

In [154]:
import torch

def get_original_indices(flat_index, shape):
    """
    Retrieve the original multidimensional indices of an element in an n-dimensional tensor
    given its index in the flattened version.

    Parameters:
    - flat_index: An integer representing the index of the element in the flattened tensor.
    - shape: The original shape of the tensor as a tuple or torch.Size object.

    Returns:
    - A list of indices representing the position of the element in the original tensor.
    """
    indices = []

    # Calculate the cumulative product of the tensor dimensions in reverse order
    # This will be used to determine the contribution of each dimension to the flat index
    cumprod = torch.tensor(shape).flip(dims=[0]).cumprod(0).flip(dims=[0])

    # Iterate through dimensions to compute the indices
    for i in range(len(shape)):
        if i == 0:
            # For the first dimension, simply divide by the cumulative product of the remaining dimensions
            idx = flat_index // cumprod[i]
        else:
            # For subsequent dimensions, calculate the remainder from the previous division
            # and divide by the cumulative product of the remaining dimensions
            idx = (flat_index % cumprod[i-1]) // cumprod[i]
        indices.append(idx.item())

    return indices

# Example usage
a = torch.randn(1, 3, 6, 15, 3)
b, c, d, h, w = a.shape 
n = b*c*d*h*w
a = torch.arange(n).reshape(a.shape)
# flat_index = torch.tensor([123])  # Example flat index
flat_index = a.max()-2

original_indices = get_original_indices(flat_index, a.shape)
print("Original indices:", original_indices)

Original indices: [0, 0, 2, 5, 14]


In [138]:
def _topk_4d(tensor_orig, k=100):
    b, c, d, h, w = tensor_orig.shape  # c will be one
    print( tensor_orig.shape)
    idxs = torch.topk(tensor_orig.view(b, -1), k, dim=-1).indices
    idxs = idxs.unsqueeze(1)
    print(idxs.shape)

    indices = torch.cat([(idxs // (h*d*w)), (idxs%(c*d*h*w))%d, (idxs %(d*h*w))%h, idxs % w], 1)
    print(indices.shape)
    indices = indices.transpose(1, 2).view(b, k, 4)
    return indices


top4 = _topk_4d(a, 5)
print(top4.shape,"\n",top4,"\n")
b = torch.arange(a.size(0)).unsqueeze(1)
print(a[b,top4[b,:,0],top4[b,:,1],top4[b,:,2], top4[b,:,3]])

b = a[:]
for i in range(5):
    print(b.max())
    b =b[b!=b.max()]



torch.Size([1, 3, 6, 15, 3])
torch.Size([1, 1, 5])
torch.Size([1, 4, 5])
torch.Size([1, 5, 4]) 
 tensor([[[ 2,  5, 14,  2],
         [ 2,  4, 13,  1],
         [ 2,  3, 12,  0],
         [ 2,  2, 11,  2],
         [ 2,  1, 10,  1]]]) 

tensor([[[809, 760, 711, 665, 616]]])
tensor(809)
tensor(808)
tensor(807)
tensor(806)
tensor(805)
